<a href="https://colab.research.google.com/github/shiling2007/Python-/blob/main/cvxpy_scipy_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://machinelearningmastery.com/function-optimization-with-scipy/

from IPython.core.display import display, HTML, Image
display(HTML("<style>.container { width:100% !important; }</style>"))
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
# from google.colab import drive
# drive.mount('/content/drive')
# from google.colab import files
# files.download('/content/drive/MyDrive/Colab Notebooks/Lease Payment Formula.ipynb') 
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.5g" % x))
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 999)
pd.set_option("max_colwidth", 500)
# try:
#  device_name = os.environ['COLAB_TPU_ADDR']
#  TPU_ADDRESS = 'grpc://' + device_name
#  print('Found TPU at: {}'.format(TPU_ADDRESS))
# except KeyError:
#  print('TPU not found')
%load_ext autoreload
%autoreload 2

In [2]:
# Generate data for control problem.
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.5g" % x))
np.random.seed(2)
n = 2
m = 1
T = 30

# A = np.eye(n) + np.random.randn(n,n)
A = np.array([[1, 0.1],[0,1]])
B = np.random.randn(n,m)
x_0 = np.random.randn(n)
x_T = np.random.randn(n)

# Form and solve control problem.
import cvxpy as cp

x = cp.Variable((n, T+1))
u = cp.Variable((m, T))
umin=0.5
umax=1
cost = 0
constr = []
for t in range(T):
    cost += cp.sum_squares(x[:,t+1]) + cp.sum_squares(u[:,t])
    constr += [x[:,t+1] == A@x[:,t] + B@u[:,t]]
# sums problem objectives and concatenates constraints.
constr += [ x[:,0] == x_0]
constr += [ u<=umax ]
constr += [ u>=umin]


problem = cp.Problem(cp.Minimize(cost), constr)
%time problem.solve(solver=cp.ECOS)
print(x.value)
print(u.value)
print(cost.value)

CPU times: user 387 ms, sys: 11.4 ms, total: 399 ms
Wall time: 529 ms
[[-2.1362 -2.1805 -2.2277 -2.2777 -2.3305 -2.3861 -2.4445 -2.5057 -2.5698 -2.6366 -2.7063 -2.7788 -2.8541 -2.9322 -3.0131 -3.0969 -3.1834 -3.2728 -3.365 -3.46 -3.5578 -3.6584 -3.7618 -3.8681 -3.9771 -4.089 -4.2037 -4.3212 -4.4415 -4.5646 -4.6906]
 [1.6403 1.6121 1.584 1.5559 1.5277 1.4996 1.4715 1.4433 1.4152 1.3871 1.3589 1.3308 1.3027 1.2745 1.2464 1.2183 1.1901 1.162 1.1339 1.1057 1.0776 1.0495 1.0213 0.9932 0.96507 0.93694 0.9088 0.88067 0.85254 0.8244 0.79627]]
[[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]]
385.8203997743301


In [3]:
#use functions
from scipy.optimize import minimize
# from numba import jit, float64, int64
# @jit(float64(float64[:], int64), nopython=True, parallel=True)
def dynamics(X, A, B, U, x_0):
  X[:,0]=x_0
  for t in range(T):
    X[:,t+1]=A@X[:,t]+B@U[:,t]
  # X[:,1:(T+1)]=A@X[:,0:T]+B@U[:,0:T]
  return X

def cost(x, *args):
    A, B, T, x_0, x_size, x_shape, u_shape = args
    X = x[:x_size].reshape(x_shape)
    U = x[x_size:].reshape(u_shape)
    
    X=dynamics(X, A, B, U, x_0)

    return np.sum(X[:,1:]*X[:,1:])+np.sum(U[:]*U[:])

x0 = np.zeros(shape=(n*(T+1)+m*T))

bnds=[]
for i in range(n*(T+1)+m*T):
  if i<n*(T+1):
    bnds.append((-np.inf, +np.inf))
  else:
    bnds.append((0.5, 1))

out = minimize(cost, x0, args=(A, B , T, x_0, n*(T+1), (n,T+1),  (m,T)), bounds=bnds, method='slsqp',  options={'maxiter': 100, 'ftol': 1e-6, 'iprint': 1, 'disp': False,})
out.fun

385.8204032951738

In [4]:
%timeit out = minimize(cost, x0, args=(A, B , T, x_0, n*(T+1), (n,T+1),  (m,T)), bounds=bnds, method='slsqp',  options={'maxiter': 100, 'ftol': 1e-6, 'iprint': 1, 'disp': False,})

10 loops, best of 5: 37.9 ms per loop


In [5]:
##use class
class opt_test():

    def __init__(self, *args):

        self.A, self.B, self.T, self.x_0, self.x_size, self.x_shape, self.u_shape = args  # state matrix 

        # self.Q = np.eye(2)
        # self.R = np.eye(1)


    def dynamics(self, x):
        X = x[:self.x_size].reshape(self.x_shape)
        U = x[self.x_size:].reshape(self.u_shape) 
        X[:,0]=self.x_0
        for t in range(self.T):
          X[:,t+1]=self.A@X[:,t]+self.B@U[:,t]
        return X
        

    def cost(self, x):
        X = x[:self.x_size].reshape(self.x_shape)
        U = x[self.x_size:].reshape(self.u_shape) 
    
        X=self.dynamics(x)

        return np.sum(X[:,1:]*X[:,1:])+np.sum(U[:]*U[:])
        
    def bd(self):
        bnds=[]
        for i in range(self.x_size+ self.B.shape[-1]*self.T):
          if i<self.x_size:
            bnds.append((-np.inf, +np.inf))
          else:
            bnds.append((0.5, 1))
        return bnds

    def opt(self, x0):

        res = minimize(self.cost, x0,  method='SLSQP',  bounds=self.bd(),
                         options={'maxiter': 200, 'disp': False})
        return res


In [6]:
x0 = np.zeros(shape=(n*(T+1)+m*T))
opt=opt_test(A, B , T, x_0, n*(T+1), (n,T+1),  (m,T))
res=opt.opt(x0)

In [7]:
res

     fun: 385.8204032951738
     jac: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 96.157, 93.45, 90.725, 87.981, 85.217, 82.429, 79.618, 76.78, 73.915, 71.02, 68.095, 65.137, 62.146, 59.119, 56.055, 52.954, 49.813, 46.631, 43.407, 40.14, 36.828, 33.47, 30.065, 26.611, 23.109, 19.556, 15.952, 12.295, 8.5847, 4.82])
 message: 'Optimization terminated successfully.'
    nfev: 94
     nit: 1
    njev: 1
  status: 0
 success: True
       x: array([-2.1362, -2.1805, -2.2277, -2.2777, -2.3305, -2.3861, -2.4445, -2.5057, -2.5698, -2.6366, -2.7063, -2.7788, -2.8541, -2.9322, -3.0131, -3.0969, -3.1834, -3.2728, -3.365, -3.46, -3.5578, -3.6584, -3.7618, -3.8681, -3.9771, -4.089, -4.2037, -4.3212, -4.4415, -4.5646, -4.6906, 1.6403, 1.6121, 1.584, 1.5559, 1.5277, 1.4996, 1.4715, 1.4433, 1.4152, 1.3871, 1.3589, 1.3308, 1.3027, 1.2745, 1.2464, 1.2183, 1.

In [8]:
%timeit res=opt.opt(x0)

10 loops, best of 5: 47.6 ms per loop


In [9]:
# x0 = np.zeros(shape=(n*(T+1)+m*T))
# xlist=[x_0]
# for i in range(1000):
#   opt=opt_test(A, B , T, x_0, n*(T+1), (n,T+1),  (m,T))
#   %timeit res = opt.opt(x0)
#   break
#   x_0=res.x[0:n*(T+1)].reshape(n,T+1)[:,1]
#   xlist.append(x_0)

In [10]:
##use class and dynamics constraints , need to work on more general cases for constraints of auto generation
class opt_test():

    def __init__(self, *args):

        self.A, self.B, self.T, self.x_0, self.x_size, self.x_shape, self.u_shape = args  # state matrix 

        # self.Q = np.eye(2)
        # self.R = np.eye(1)


    def dynamics(self, x):
        X = x[:self.x_size].reshape(self.x_shape)
        U = x[self.x_size:].reshape(self.u_shape) 
        X[:,0]=self.x_0
        for t in range(self.T):
          X[:,t+1]=self.A@X[:,t]+self.B@U[:,t]
        return X
        

    def cost(self, x):
        X = x[:self.x_size].reshape(self.x_shape)
        U = x[self.x_size:].reshape(self.u_shape) 
    
        X=self.dynamics(x)

        return np.sum(X[:,1:]*X[:,1:])+np.sum(U[:]*U[:])
        # return np.sum(x[(self.T+1):self.x_size]*x[(self.T+1):self.x_size])+np.sum(x[self.x_size:]*x[self.x_size:])
        
    def bd(self):
        bnds=[]
        for i in range(self.x_size+ self.B.shape[-1]*self.T):
          if i<self.x_size:
            bnds.append((-np.inf, +np.inf))
          else:
            bnds.append((0.5, 1))
        return bnds

    def constrain_list(self, x):
        constrain_list=[]
    
        for t in range(self.T):

          constrain_list.append( { 'type':'eq', 'fun': lambda x:  (x[t+1],x[t+1+(self.T+1)])-(A@(x[t],x[t+(self.T+1)]) + B@(x[2*(self.T+1)+t].reshape(1)))} )
    
        return constrain_list

    def opt(self, x0):

        res = minimize(self.cost, x0,  method='SLSQP',   bounds=self.bd(),
                       constraints = self.constrain_list(x),
                         options={'maxiter': 200, 'disp': False})
        return res
# ({ 'type':'eq', 'fun': lambda x:  (x[0+1],x[0+1+(T+1)])-(A@(x[0],x[0+(T+1)]) + B@(x[n*(T+1)+0].reshape(1)))},
#                                       { 'type':'eq', 'fun': lambda x:  (x[1+1],x[1+1+(T+1)])-(A@(x[1],x[1+(T+1)]) + B@(x[n*(T+1)+1].reshape(1)))},
#                                       { 'type':'eq', 'fun': lambda x:  (x[2+1],x[2+1+(T+1)])-(A@(x[2],x[2+(T+1)]) + B@(x[n*(T+1)+2].reshape(1)))},
#                                       { 'type':'eq', 'fun': lambda x:  (x[3+1],x[3+1+(T+1)])-(A@(x[3],x[3+(T+1)]) + B@(x[n*(T+1)+3].reshape(1)))},)

In [11]:
x0 = np.zeros(shape=(n*(T+1)+m*T))
opt=opt_test(A, B , T, x_0, n*(T+1), (n,T+1),  (m,T))
res2=opt.opt(x0)

In [12]:
res2

     fun: 385.8204032951738
     jac: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 96.157, 93.45, 90.725, 87.981, 85.217, 82.429, 79.618, 76.78, 73.915, 71.02, 68.095, 65.137, 62.146, 59.119, 56.055, 52.954, 49.813, 46.631, 43.407, 40.14, 36.828, 33.47, 30.065, 26.611, 23.109, 19.556, 15.952, 12.295, 8.5847, 4.82])
 message: 'Singular matrix C in LSQ subproblem'
    nfev: 94
     nit: 1
    njev: 1
  status: 6
 success: False
       x: array([-2.1362, -2.1805, -2.2277, -2.2777, -2.3305, -2.3861, -2.4445, -2.5057, -2.5698, -2.6366, -2.7063, -2.7788, -2.8541, -2.9322, -3.0131, -3.0969, -3.1834, -3.2728, -3.365, -3.46, -3.5578, -3.6584, -3.7618, -3.8681, -3.9771, -4.089, -4.2037, -4.3212, -4.4415, -4.5646, -4.6906, 1.6403, 1.6121, 1.584, 1.5559, 1.5277, 1.4996, 1.4715, 1.4433, 1.4152, 1.3871, 1.3589, 1.3308, 1.3027, 1.2745, 1.2464, 1.2183, 1.1

In [13]:
%timeit res2=opt.opt(x0)

10 loops, best of 5: 54.1 ms per loop
